In [2]:
import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9 # 使⽤45%記憶體
# set_session(tf.Session()) 

In [3]:
import numpy as np
from os import listdir
from random import shuffle
from PIL import Image


image_path = ["../DeepLearning/kermany2018/NewSize/train/CNV", 
              "../DeepLearning/kermany2018/NewSize/train/DME",
              "../DeepLearning/kermany2018/NewSize/train/DRUSEN",
              "../DeepLearning/kermany2018/NewSize/train/NORMAL"]
#image_path = ["./data/A", "./data/B"]

def preprocess_to_data_file(data_dir_list, ratio=0.8):

    total_list = [] 
   
    '''
        create all images into (image path, label) format and write to total_data.txt 
    ''' 
    with open('total_data.txt', 'w') as f:
        for index, data_dir in enumerate(data_dir_list):
            for filename in listdir(data_dir):
                #print("{} {}".format(data_dir_list[index]+'/'+filename, index))
                f.write('{} {}\n'.format(data_dir_list[index]+'/'+filename.replace(' ',''), index))
                total_list.append(data_dir_list[index]+'/'+filename.replace(' ','')+' '+str(index))

    #print(total_list)
    '''
        shuffle total_list
    ''' 
    shuffle(total_list)

    '''
        split total_list to train_list and test_list. 
        write train_list/test_list into train_data.txt/test_data.txt
    ''' 
    train_list = total_list[:int(ratio*len(total_list))]
    test_list = total_list[int(ratio*len(total_list)):]
    print(len(train_list),len(test_list))

    with open('train_data.txt', 'w') as f:
        for i in train_list:
            f.write(i+'\n')

    with open('test_data.txt', 'w') as f:
        for i in test_list:
            f.write(i+'\n')



preprocess_to_data_file(image_path, 0.8)


66787 16697


In [5]:
import numpy as np
from random import shuffle
from PIL import Image
import tensorflow as tf
import os
tf.reset_default_graph()

def one_hot_encoding(label):

    '''
    change label id to one-hot encoding
    '''

    values = np.asarray(label)
    n_class = 4
    encoding_result = np.eye(n_class)[values]
    return encoding_result 



def load_data(file_path):

    '''
    load all of training data
    '''

    with open(file_path, "r") as lines:
        data_list = []
        for line in lines:
            data_list.append(line.replace('\n',''))

        shuffle(data_list) 

    data_path = []
    data_label = []
    for data in data_list:
        data_path.append(data.split(' ')[0])
        data_label.append(int(data.split(' ')[1]))

    return data_path, data_label


def load_batch_data(data_path, labels):
    

    batch_data = []
    for index, im in enumerate(data_path): 
        raw_image = Image.open(im)
        ### please notice this line if image have already been normalized 
        resize_image = raw_image.resize((80, 80))
        normalized_image = np.asarray(resize_image)/255.0
        normalized_image = normalized_image.reshape(80,80,1)
        batch_data.append(normalized_image)

    batch_label = one_hot_encoding(labels)


    batch_data = np.asarray(batch_data, np.float32)
    batch_label = np.asarray(batch_label, np.float32)
    return batch_data, batch_label



print('loading image path......')
train_data, train_label = load_data("train_data.txt")
test_data, test_label = load_data("test_data.txt")


print('number of train image is {}'.format(len(train_data)))
print('number of test image is {}'.format(len(test_data)))


'''
set network parameters 

'''
# 162*212
image_size_width = 80
image_size_height = 80
num_labels = 4 # 4種
num_channels = 1 # 黑白
batch_size = 256
kernel_size = 3
num_steps = 15001

'''
create CNN model
'''

x = tf.placeholder(tf.float32, [None, image_size_height, image_size_width, num_channels],name='x')
y = tf.placeholder(tf.float32, [None, num_labels],name='y')

# initial variables
layer1_weights = tf.Variable(tf.truncated_normal([kernel_size, kernel_size, num_channels, 32], stddev=0.1))
layer1_biases = tf.Variable(tf.zeros([32]))
layer2_weights = tf.Variable(tf.truncated_normal([kernel_size, kernel_size, 32, 64], stddev=0.1))
layer2_biases = tf.Variable(tf.constant(1.0, shape=[64]))
layer3_weights = tf.Variable(tf.truncated_normal([kernel_size, kernel_size, 64, 128], stddev=0.1))
layer3_biases = tf.Variable(tf.constant(1.0, shape=[128]))

layer4_weights = tf.Variable(tf.truncated_normal([12800, 1024], stddev=0.1))
layer4_biases = tf.Variable(tf.constant(1.0, shape=[1024]))
layer5_weights = tf.Variable(tf.truncated_normal([1024, num_labels], stddev=0.1))
layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

# CNN model detail
def model(input_image):
    conv1 = tf.nn.conv2d(input_image, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv1 + layer1_biases)
    pool1 = tf.nn.max_pool(hidden1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    conv2 = tf.nn.conv2d(hidden1, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden2 = tf.nn.relu(conv2 + layer2_biases)
    pool2 = tf.nn.max_pool(hidden2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    conv3 = tf.nn.conv2d(hidden2, layer3_weights, [1, 2, 2, 1], padding='SAME')
    hidden3 = tf.nn.relu(conv3 + layer3_biases)
    shape = hidden3.get_shape().as_list()

    reshape = tf.reshape(hidden3, [-1, shape[1] * shape[2] * shape[3]])

    hidden = tf.nn.relu(tf.matmul(reshape, layer4_weights) + layer4_biases)
    
    return tf.add(tf.matmul(hidden, layer5_weights),layer5_biases,name="logis")

# build model
logits = model(x)

# define cost
# with tf.name_scope('Loss'):
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits))

# optimization
# with tf.name_scope('Train'):
optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)
# show prediction result
prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

# 初始化 Graph
# init = tf.global_variables_initializer()
# sess = tf.Session()

# 將視覺化輸出
# writer = tf.summary.FileWriter("TensorBoard/", graph = sess.graph)

# 開始運算
# saver = tf.train.Saver()

with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    print('start training......')
    for step in range(num_steps):
        offset = (step * batch_size) % (len(train_data) - batch_size)
        batch_data_path = train_data[offset:(offset + batch_size)]
        batch_label_path = train_label[offset:(offset + batch_size)]

        train_batch_data, train_batch_labels = load_batch_data(batch_data_path, batch_label_path)


        feed_dict = { x: train_batch_data, y: train_batch_labels}
        _, l, train_accuracy_ = sess.run([optimizer, loss, accuracy], feed_dict=feed_dict)


        if (step % 100 == 0):
            saver.save(sess, "train_model/model.ckpt")
            print('step={}, loss={}, accuracy={}'.format(step, l, train_accuracy_))
            test_batch_data, test_batch_labels = load_batch_data(test_data[:2000], test_label[:2000])
            feed_dict = { x: test_batch_data, y: test_batch_labels}
            test_accuracy_ = sess.run(accuracy, feed_dict=feed_dict)
            print('test accuracy = {}'.format(test_accuracy_))
    
    #print('start testing......')

loading image path......
number of train image is 66787
number of test image is 16697
start training......
step=0, loss=25.995214462280273, accuracy=0.1171875
test accuracy = 0.43549999594688416
step=100, loss=1.5034153461456299, accuracy=0.33203125
test accuracy = 0.30149999260902405
step=200, loss=2.3095462322235107, accuracy=0.43359375
test accuracy = 0.4465000033378601
step=300, loss=2.0140233039855957, accuracy=0.43359375
test accuracy = 0.3154999911785126
step=400, loss=1.2382534742355347, accuracy=0.46484375
test accuracy = 0.4675000011920929
step=500, loss=2.731905937194824, accuracy=0.4921875
test accuracy = 0.4724999964237213
step=600, loss=1.7527737617492676, accuracy=0.30859375
test accuracy = 0.4065000116825104
step=700, loss=1.170323133468628, accuracy=0.54296875
test accuracy = 0.492000013589859
step=800, loss=1.1742198467254639, accuracy=0.55078125
test accuracy = 0.5065000057220459
step=900, loss=2.805194854736328, accuracy=0.1875
test accuracy = 0.4494999945163727
ste

test accuracy = 0.7754999995231628
step=9100, loss=0.5460309982299805, accuracy=0.76953125
test accuracy = 0.7519999742507935
step=9200, loss=0.4035431444644928, accuracy=0.84375
test accuracy = 0.7689999938011169
step=9300, loss=0.4135703444480896, accuracy=0.83203125
test accuracy = 0.7749999761581421
step=9400, loss=0.5283043384552002, accuracy=0.80859375
test accuracy = 0.7770000100135803
step=9500, loss=0.45055079460144043, accuracy=0.83984375
test accuracy = 0.7799999713897705
step=9600, loss=0.5050524473190308, accuracy=0.80078125
test accuracy = 0.7864999771118164
step=9700, loss=0.40723755955696106, accuracy=0.83984375
test accuracy = 0.7835000157356262
step=9800, loss=0.4916848838329315, accuracy=0.80078125
test accuracy = 0.7754999995231628
step=9900, loss=0.4205305874347687, accuracy=0.8515625
test accuracy = 0.784500002861023
step=10000, loss=0.4520394504070282, accuracy=0.79296875
test accuracy = 0.7730000019073486
step=10100, loss=0.4645250141620636, accuracy=0.81640625


In [ ]:
# def load_batch_data(data_path):
    

#     batch_data = []
#     for index, im in enumerate(data_path): 
raw_image = Image.open('kermany2018/NewSize/test/CNV/CNV-53018-1.jpeg')
### please notice this line if image have already been normalized 
resize_image = raw_image.resize((80, 80))
normalized_image = np.asarray(resize_image)/255.0
normalized_image = normalized_image.reshape(80,80,1)
#         batch_data.append(normalized_image)

#     batch_label = one_hot_encoding(labels)


batch_data = np.asarray(normalized_image, np.float32)
batch_data=batch_data.reshape(1,80,80,1)
#     batch_label = np.asarray(batch_label, np.float32)
#         return batch_data
print(batch_data.shape)

In [ ]:
tf.reset_default_graph()
# saver = tf.train.import_meta_graph('train_model/model.ckpt.meta')


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess,"train_model/model.ckpt")    
#     test_accuracy_ = sess.run(logits, feed_dict={x:batch_data})
    test_batch_data, test_batch_labels = load_batch_data(test_data[:2000], test_label[:2000])
    feed_dict = { x: test_batch_data, y: test_batch_labels}
    test_accuracy_ = sess.run(accuracy, feed_dict=feed_dict)
    print('test accuracy = {}'.format(test_accuracy_))
# sess.run(logits, feed_dict={x:batch_data}) 